In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly as pys
import ipywidgets as widgets
from ipywidgets import Layout

import _global_scripts as gs

## Purpose

## Inputs

In [2]:
obs_access_dist_df = pd.read_csv(r"_data/E2.6/OBS_Access_Egress_Distances_v2.csv")

obs_df = pd.read_csv(r"C:\Users\bhereth\Documents\2019 Final Weighted UTA OD Data - 2022-04-05 - processed.csv", low_memory=False)
obs_crt_df = obs_df[(obs_df['Linked_Mode_txt']=='CRT')]

# all HBC are peak period
obs_crt_df.loc[obs_crt_df['Purp5_text'] == 'HBC', 'PK_OK'] = 'PK'

# remove HBSch
obs_crt_df = obs_crt_df[obs_crt_df['Purp5_text']!='HBSch']

obs_crt_df

,id,date_completed,final_route_surveyed_code,final_route_surveyed,direction,mode,purpose,resident_or_visitor_code,home_or_hotel_address,home_or_hotel_city,...,FirstOn_lat,FirstOn_lon,LastOff_lat,LastOff_lon,p_Stop_lat,p_Stop_lon,a_Stop_lat,a_Stop_lon,unlinked_weight_adj,linked_weight_adj
3,30.0,9-Sep-19,SLC_1_703_00,TRAX RED LINE TO DAYBREAK PARKWAY STATION,0.0,1.0,2.0,1.0,262 E Whitlock Ave,South Salt Lake,...,40.724091,-111.896892,40.280140,-111.725489,40.724091,-111.896892,40.280140,-111.725489,11.440,5.720
5,32.0,9-Sep-19,SLC_1_701_00,TRAX BLUE LINE TO DRAPER TOWN CENTER STATION,0.0,1.0,1.0,1.0,10404 Avondale Dr,Cedar Hills,...,40.374774,-111.820649,40.767452,-111.891101,40.374774,-111.820649,40.767452,-111.891101,18.136,9.068
17,61.0,9-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,4.0,2.0,10333 S Jordan Gateway,South Jordan,...,40.563155,-111.900753,40.769344,-111.901146,40.563155,-111.900753,40.769344,-111.901146,3.265,1.633
46,212.0,11-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,1.0,1.0,W Coyote Cir,Saratoga Springs,...,40.425196,-111.896354,40.763855,-111.891071,40.425196,-111.896354,40.763855,-111.891071,9.312,4.656
58,236.0,11-Sep-19,SLC_1_704_01,TRAX GREEN LINE TO AIRPORT STATION,1.0,1.0,3.0,1.0,622 23rd St,Ogden,...,41.224285,-111.980631,40.783280,-111.980172,41.224285,-111.980631,40.783280,-111.980172,7.603,3.801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13403,22722.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,641 1850 N St,Ogden,...,40.763855,-111.891071,41.224285,-111.980631,41.224285,-111.980631,40.763855,-111.891071,11.951,5.976
13409,22737.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,185 E 1050 N,Bountiful,...,40.768217,-111.891081,40.880457,-111.903151,40.880457,-111.903151,40.768217,-111.891081,17.630,8.815
13410,22738.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,2803 W 1475 N,Layton,...,40.768217,-111.891081,41.094769,-112.013807,41.094769,-112.013807,40.768217,-111.891081,11.951,5.976
13414,22744.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,W 4800 S,Roy,...,40.772532,-111.905124,41.188757,-112.039378,41.188757,-112.039378,40.772532,-111.905124,11.238,11.238


In [3]:
def weighted_quantile(data, q, weights, factor):
    factored_weights = weights * factor
    rounded_weights = np.round(factored_weights).astype(int)
    repeat_data = np.repeat(data,rounded_weights).astype(float)

    return np.quantile(repeat_data, q)

def weighted_average(data, weights):
    return np.average(data, weights=weights)

## Automated Passenger Counters

In [ ]:
apc_df = pd.read_csv(r"_data\E2.A3\Ridership Trax Sept to Nov 20191206 FrontRunner.csv")
# Trim spaces from headers
apc_df.columns = apc_df.columns.str.strip()

# Trim spaces from all data fields
apc_df = apc_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Identify numeric columns and remove commas only from them
for col in apc_df.select_dtypes(include=['object']):  # Only process object (string) columns
    if apc_df[col].str.replace(',', '', regex=True).str.isnumeric().all():  # Check if all values are numeric
        apc_df[col] = apc_df[col].str.replace(',', '', regex=True).astype(float)

apc_df['AvgDailyAlightings'] = apc_df['AvgDailyAlightings'].astype(int)

apc_df['Station'] = apc_df['Station'].str.replace(' Departure','')

apc_df.loc[apc_df['Station']=='Salt Lake Central', 'Station'] = 'Salt Lake Central Station'

apc_df = apc_df[apc_df['Service']=='WKD']
apc_df

FileNotFoundError: [Errno 2] No such file or directory: '_data\\E2.15\\Ridership Trax Sept to Nov 20191206 FrontRunner.csv'

In [ ]:
apc_df = apc_df[apc_df['Service']=='WKD']
apc_df

,Service,Route,DirName,FX_DirNum,Sequence,Station,StopID,TimePeriod,AvgDailyBoardings,AvgDailyAlightings
120,WKD,750,NORTHBOUND,0,1,Provo Central Station,23073,1-AM Peak,975,98
121,WKD,750,NORTHBOUND,0,1,Provo Central Station,23073,2-Midday,531,69
122,WKD,750,NORTHBOUND,0,1,Provo Central Station,23073,3-PM Peak,576,90
123,WKD,750,NORTHBOUND,0,1,Provo Central Station,23073,4-Evening,194,41
124,WKD,750,NORTHBOUND,0,2,Orem Central Station,23076,1-AM Peak,568,38
...,...,...,...,...,...,...,...,...,...,...
235,WKD,750,SOUTHBOUND,1,16,Orem Central Station,23076,4-Evening,6,87
236,WKD,750,SOUTHBOUND,1,17,Provo Central Station,23073,1-AM Peak,10,498
237,WKD,750,SOUTHBOUND,1,17,Provo Central Station,23073,2-Midday,15,751
238,WKD,750,SOUTHBOUND,1,17,Provo Central Station,23073,3-PM Peak,8,612


In [ ]:
apc_board_alight_df = apc_df.groupby(['Station','DirName','TimePeriod'], as_index=False).agg(Boardings=('AvgDailyBoardings','sum'),Alightings=('AvgDailyAlightings','sum'))

apc_board_alight_df['Direction'] = apc_board_alight_df['DirName'].replace({"NORTHBOUND": 'Northbound', "SOUTHBOUND": 'Southbound'})
apc_board_alight_df['Period'] = apc_board_alight_df['TimePeriod']

apc_board_alight_df['Source'] = 'APC'


apc_board_alight_df = apc_board_alight_df[['Source','Station','Direction','Period','Boardings','Alightings']]

apc_board_alight_df

,Source,Station,Direction,Period,Boardings,Alightings
0,APC,American Fork Station,Northbound,1-AM Peak,414,26
1,APC,American Fork Station,Northbound,2-Midday,94,68
2,APC,American Fork Station,Northbound,3-PM Peak,92,98
3,APC,American Fork Station,Northbound,4-Evening,24,23
4,APC,American Fork Station,Southbound,1-AM Peak,155,54
...,...,...,...,...,...,...
115,APC,Woods Cross Station,Northbound,4-Evening,9,28
116,APC,Woods Cross Station,Southbound,1-AM Peak,249,77
117,APC,Woods Cross Station,Southbound,2-Midday,85,53
118,APC,Woods Cross Station,Southbound,3-PM Peak,65,34


In [ ]:
apc_board_alight_df[['Boardings','Alightings']].sum()

Boardings     18822
Alightings    18849
dtype: int64

## Comparison of Boardings and OBS

In [ ]:
obs_crt_df

,id,date_completed,final_route_surveyed_code,final_route_surveyed,direction,mode,purpose,resident_or_visitor_code,home_or_hotel_address,home_or_hotel_city,...,FirstOn_lat,FirstOn_lon,LastOff_lat,LastOff_lon,p_Stop_lat,p_Stop_lon,a_Stop_lat,a_Stop_lon,unlinked_weight_adj,linked_weight_adj
3,30.0,9-Sep-19,SLC_1_703_00,TRAX RED LINE TO DAYBREAK PARKWAY STATION,0.0,1.0,2.0,1.0,262 E Whitlock Ave,South Salt Lake,...,40.724091,-111.896892,40.280140,-111.725489,40.724091,-111.896892,40.280140,-111.725489,11.440,5.720
5,32.0,9-Sep-19,SLC_1_701_00,TRAX BLUE LINE TO DRAPER TOWN CENTER STATION,0.0,1.0,1.0,1.0,10404 Avondale Dr,Cedar Hills,...,40.374774,-111.820649,40.767452,-111.891101,40.374774,-111.820649,40.767452,-111.891101,18.136,9.068
17,61.0,9-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,4.0,2.0,10333 S Jordan Gateway,South Jordan,...,40.563155,-111.900753,40.769344,-111.901146,40.563155,-111.900753,40.769344,-111.901146,3.265,1.633
46,212.0,11-Sep-19,SLC_1_701_01,TRAX BLUE LINE TO SALT LAKE CENTRAL STATION,1.0,1.0,1.0,1.0,W Coyote Cir,Saratoga Springs,...,40.425196,-111.896354,40.763855,-111.891071,40.425196,-111.896354,40.763855,-111.891071,9.312,4.656
58,236.0,11-Sep-19,SLC_1_704_01,TRAX GREEN LINE TO AIRPORT STATION,1.0,1.0,3.0,1.0,622 23rd St,Ogden,...,41.224285,-111.980631,40.783280,-111.980172,41.224285,-111.980631,40.783280,-111.980172,7.603,3.801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13403,22722.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,641 1850 N St,Ogden,...,40.763855,-111.891071,41.224285,-111.980631,41.224285,-111.980631,40.763855,-111.891071,11.951,5.976
13409,22737.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,185 E 1050 N,Bountiful,...,40.768217,-111.891081,40.880457,-111.903151,40.880457,-111.903151,40.768217,-111.891081,17.630,8.815
13410,22738.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,2803 W 1475 N,Layton,...,40.768217,-111.891081,41.094769,-112.013807,41.094769,-112.013807,40.768217,-111.891081,11.951,5.976
13414,22744.0,11-Dec-19,SLC_1_750_00,FRONTRUNNER TO OGDEN STATION,0.0,2.0,1.0,1.0,W 4800 S,Roy,...,40.772532,-111.905124,41.188757,-112.039378,41.188757,-112.039378,40.772532,-111.905124,11.238,11.238


In [ ]:
obs_df = pd.read_csv(r"C:\Users\bhereth\Documents\2019 Final Weighted UTA OD Data - 2022-04-05 - processed.csv", low_memory=False)
obs_crt_df = obs_df[(obs_df['final_route_surveyed'].isin(['FRONTRUNNER TO OGDEN STATION','FRONTRUNNER TO PROVO CENTRAL STATION']))].copy()

# all HBC are peak period
obs_crt_df.loc[obs_crt_df['Purp5_text'] == 'HBC', 'PK_OK'] = 'PK'
obs_crt_df['Direction'] = obs_crt_df['direction'].replace({0: 'Northbound', 1: 'Southbound'})

# remove HBSch
#obs_crt_df = obs_crt_df[obs_crt_df['Purp5_text']!='HBSch']

obs_crt_board_df = obs_crt_df.copy()
obs_crt_alight_df = obs_crt_df.copy()

obs_crt_board_df['Station'] = obs_crt_df['stop_on_addr'].str.title()#.str.replace(' Station', '', regex=False)
obs_crt_alight_df['Station'] = obs_crt_df['stop_off_addr'].str.title()#.str.replace(' Station', '', regex=False)

# used unlinked_weight and not unlinked_weight_adj, since unadjusted matches boardings data
obs_crt_board_df = obs_crt_board_df[['Direction','Period','Station','unlinked_weight']]
obs_crt_alight_df = obs_crt_alight_df[['Direction','Period','Station','unlinked_weight']]

obs_crt_board_df = obs_crt_board_df.groupby(['Station','Direction','Period'], as_index=False).agg(Boardings=('unlinked_weight','sum'))
obs_crt_alight_df = obs_crt_alight_df.groupby(['Station','Direction','Period'], as_index=False).agg(Alightings=('unlinked_weight','sum'))

display(obs_crt_board_df, obs_crt_alight_df)


,Station,Direction,Period,Boardings
0,American Fork Station,Northbound,AM,364.07190
1,American Fork Station,Northbound,EV,12.20608
2,American Fork Station,Northbound,MD,67.28517
3,American Fork Station,Northbound,PM,67.08185
4,American Fork Station,Southbound,AM,151.99617
...,...,...,...,...
106,Woods Cross Station,Northbound,PM,39.53233
107,Woods Cross Station,Southbound,AM,224.03874
108,Woods Cross Station,Southbound,EV,9.45552
109,Woods Cross Station,Southbound,MD,63.75463


,Station,Direction,Period,Alightings
0,American Fork Station,Northbound,EV,55.58292
1,American Fork Station,Northbound,MD,81.54336
2,American Fork Station,Northbound,PM,171.84756
3,American Fork Station,Southbound,AM,85.24807
4,American Fork Station,Southbound,EV,63.25453
...,...,...,...,...
103,Woods Cross Station,Northbound,PM,93.42070
104,Woods Cross Station,Southbound,AM,88.29654
105,Woods Cross Station,Southbound,EV,16.74490
106,Woods Cross Station,Southbound,MD,50.49468


In [ ]:
obs_board_alight_df = pd.merge(obs_crt_board_df,obs_crt_alight_df,on=['Station','Direction','Period'],how='outer').fillna(0)
obs_board_alight_df

,Station,Direction,Period,Boardings,Alightings
0,American Fork Station,Northbound,AM,364.07190,0.00000
1,American Fork Station,Northbound,EV,12.20608,55.58292
2,American Fork Station,Northbound,MD,67.28517,81.54336
3,American Fork Station,Northbound,PM,67.08185,171.84756
4,American Fork Station,Southbound,AM,151.99617,85.24807
...,...,...,...,...,...
115,Woods Cross Station,Northbound,PM,39.53233,93.42070
116,Woods Cross Station,Southbound,AM,224.03874,88.29654
117,Woods Cross Station,Southbound,EV,9.45552,16.74490
118,Woods Cross Station,Southbound,MD,63.75463,50.49468


In [ ]:
obs_board_alight_df['Period'] = obs_board_alight_df['Period'].replace({'AM': '1-AM Peak', 'MD': '2-Midday', 'PM':'3-PM Peak', 'EV':'4-Evening'})
obs_board_alight_df['Source'] = 'OBS'
obs_board_alight_df = obs_board_alight_df[['Source','Station','Direction','Period','Boardings','Alightings']]
obs_board_alight_df

,Source,Station,Direction,Period,Boardings,Alightings
0,OBS,American Fork Station,Northbound,1-AM Peak,364.07190,0.00000
1,OBS,American Fork Station,Northbound,4-Evening,12.20608,55.58292
2,OBS,American Fork Station,Northbound,2-Midday,67.28517,81.54336
3,OBS,American Fork Station,Northbound,3-PM Peak,67.08185,171.84756
4,OBS,American Fork Station,Southbound,1-AM Peak,151.99617,85.24807
...,...,...,...,...,...,...
115,OBS,Woods Cross Station,Northbound,3-PM Peak,39.53233,93.42070
116,OBS,Woods Cross Station,Southbound,1-AM Peak,224.03874,88.29654
117,OBS,Woods Cross Station,Southbound,4-Evening,9.45552,16.74490
118,OBS,Woods Cross Station,Southbound,2-Midday,63.75463,50.49468


In [ ]:
obs_board_alight_df[['Boardings','Alightings']].sum()

Boardings     18824.87893
Alightings    18824.87893
dtype: float64

# Combine

In [ ]:
board_alight_df = pd.concat([apc_board_alight_df,obs_board_alight_df])
board_alight_df

,Source,Station,Direction,Period,Boardings,Alightings
0,APC,American Fork Station,Northbound,1-AM Peak,414.00000,26.00000
1,APC,American Fork Station,Northbound,2-Midday,94.00000,68.00000
2,APC,American Fork Station,Northbound,3-PM Peak,92.00000,98.00000
3,APC,American Fork Station,Northbound,4-Evening,24.00000,23.00000
4,APC,American Fork Station,Southbound,1-AM Peak,155.00000,54.00000
...,...,...,...,...,...,...
115,OBS,Woods Cross Station,Northbound,3-PM Peak,39.53233,93.42070
116,OBS,Woods Cross Station,Southbound,1-AM Peak,224.03874,88.29654
117,OBS,Woods Cross Station,Southbound,4-Evening,9.45552,16.74490
118,OBS,Woods Cross Station,Southbound,2-Midday,63.75463,50.49468


In [ ]:
sequence_df = apc_df[apc_df['DirName']=='SOUTHBOUND'].copy()
sequence_df['Sequence'] = sequence_df['Sequence']-1
sequence_df['Station_Sequence'] = sequence_df['Sequence'].astype(str).str.zfill(2) + '-' + sequence_df['Station']
sequence_df = sequence_df[['Station','Station_Sequence']].drop_duplicates()
sequence_df

,Station,Station_Sequence
180,Ogden Station,01-Ogden Station
184,Roy Station,02-Roy Station
188,Clearfield Station,03-Clearfield Station
192,Layton Station,04-Layton Station
196,Farmington Station,05-Farmington Station
200,Woods Cross Station,06-Woods Cross Station
204,North Temple Station,08-North Temple Station
208,Salt Lake Central Station,09-Salt Lake Central Station
212,Murray Central Station,10-Murray Central Station
216,South Jordan Station,11-South Jordan Station


In [ ]:
board_alight_sequence_df = pd.merge(board_alight_df, sequence_df, on=['Station'], how='outer')
board_alight_sequence_df = board_alight_sequence_df[['Station','Source','Station_Sequence','Direction','Period','Boardings','Alightings']]
board_alight_sequence_df['Board+Alight / 2'] = board_alight_sequence_df['Boardings'] / 2 + board_alight_sequence_df['Alightings'] / 2
board_alight_sequence_df

,Station,Source,Station_Sequence,Direction,Period,Boardings,Alightings,Board+Alight / 2
0,American Fork Station,APC,14-American Fork Station,Northbound,1-AM Peak,414.00000,26.00000,220.000000
1,American Fork Station,APC,14-American Fork Station,Northbound,2-Midday,94.00000,68.00000,81.000000
2,American Fork Station,APC,14-American Fork Station,Northbound,3-PM Peak,92.00000,98.00000,95.000000
3,American Fork Station,APC,14-American Fork Station,Northbound,4-Evening,24.00000,23.00000,23.500000
4,American Fork Station,APC,14-American Fork Station,Southbound,1-AM Peak,155.00000,54.00000,104.500000
...,...,...,...,...,...,...,...,...
235,Woods Cross Station,OBS,06-Woods Cross Station,Northbound,3-PM Peak,39.53233,93.42070,66.476515
236,Woods Cross Station,OBS,06-Woods Cross Station,Southbound,1-AM Peak,224.03874,88.29654,156.167640
237,Woods Cross Station,OBS,06-Woods Cross Station,Southbound,4-Evening,9.45552,16.74490,13.100210
238,Woods Cross Station,OBS,06-Woods Cross Station,Southbound,2-Midday,63.75463,50.49468,57.124655


In [ ]:
board_alight_sequence_df.to_csv('apc-obs-board-alight/boardings.csv', index=False)
print('Exported')

Exported
